# Data Cleaning and Merging

In the following script, I am merging data from various sources to create one master data set that includes all the variables of interest to my analysis. Some key differences to note about Delhi is that the last Census of India occured in 2011, while multiple re-deliminations to municipal electoral ward boundaries were made in this decade, meaning boundary “vintages” do not line up cleanly across datasets, and I could not use the official Delhi GeoJSON because those were 2025/2026 boundaries. ([AP News][1])

Therefore, to keep consistent geographic units, I took ward polygons from the community-compiled **DataMeet Municipal Spatial Data** repository, and as such should be taken with the errors that come with it. Note that these are **municipal wards (not census wards)**. In parallel, district boundaries were sourced from **HindustanTimesLabs** (Delhi district boundary GeoJSON), and each ward is assigned to a district using a centroid-based spatial join. ([GitHub][3])

Because ward counts and boundaries changed over time, results should be interpreted as applying to the **ward geography used in the script**, rather than “today’s” wards. For reference, the State Election Commission documents that municipal elections were conducted for **272 wards** in 2017, and the 2022 delimitation process determined **250 wards** for the unified MCD. ([Delhi Government Security][4]) 

Additionally, another Delhi-specific complication is that the “Delhi local government” is not a single municipal body: the urban area includes separate jurisdictions under the **New Delhi Municipal Council (NDMC)** and the **Delhi Cantonment Board**, which are administratively distinct from the Municipal Corporation system that most ward datasets target. ([Delhi Government][5]) As a result, some wards, especially NDMC/Cantonment-related units in my final dataset, were not matched cleanly to ward-numbered population tables, and I added these manually from the census ward level data. ([Delhi Government][5])

For public transport service, GTFS schedule tables are combined using standard GTFS identifiers (e.g., `stop_id`, `trip_id`, `route_id`), then bus stops are assigned to wards using a point-in-polygon spatial join. ([General Transit Feed Specification][6]) Bus service intensity is constructed as a standardized index of stop counts, route counts, and trip counts within each ward; this should be understood as a **constructed service index** rather than a directly published administrative statistic. Note that most GTFS based tables have a service calendar applied, but the data I downloaded for Delhi did not have times when service was not being offered, likely because the MCD does not presently have the infrastructure to keep track of this. Therefore, calendar counts based on `trip_id` represent a **proxy** for daily service rather than a fully calendar-filtered “trips per day” measure. ([General Transit Feed Specification][7])

Several predictors are computed measures derived from spatial data rather than directly observed ward totals. Metro access is measured as distance (or inverse distance) from ward centroids to metro stations; if a complete station file is not available programmatically, results may reflect the station list used. Road density is estimated from **OpenStreetMap** road networks downloaded via **OSMnx**, which relies on OSM coverage and definitions (e.g., the “drive” network) and therefore can vary with map completeness. ([networkx.org][8])

Comments and suggestions (especially for more efficient ways for finding Delhi data!) are always, always welcome. 

[1]: https://apnews.com/article/india-census-caste-politics-bihar-welfare-population-2a49a372c4f9bd79c4b0a11ed514c8a2 "India will start its delayed census next year and will ask ..."
[2]: https://projects.datameet.org/Municipal_Spatial_Data "Spatial Data of Municipalities (Maps)"
[3]: https://github.com/HindustanTimesLabs/shapefiles "HindustanTimesLabs/shapefiles"
[4]: https://sec.delhi.gov.in/sec/general-election-272-wards-three-municipal-corporation-delhi-2017-respectively-supplysale "General Election to 272 Wards of three Municipal ..."
[5]: https://des.delhi.gov.in/sites/default/files/DES/generic_multiple_files/report_on_lb_wise.pdf "Report on Sixth Economic Census: Profile of Local Bodies, ..."
[6]: https://gtfs.org/documentation/schedule/reference "General Transit Feed Specification Reference"
[7]: https://gtfs.org/getting-started/features/base "Base"
[8]: https://networkx.org/documentation/stable/auto_examples/geospatial/plot_osmnx.html "OpenStreetMap with OSMnx"
[9]: https://github.com/datameet/Municipal_Spatial_Data/issues/57 "Delhi ward data is outdated · Issue #57 - GitHub"
[10]: https://github.com/HindustanTimesLabs/shapefiles/blob/master/city/delhi/district/delhi_1997-2012_district.json "shapefiles/city/delhi/district/delhi_1997-2012_district.json ..."
[11]: https://sec.delhi.gov.in/sec/delimitation-order-2022 "Delimitation Order 2022"


The following code merges the GTFS code: 

In [2]:
import pandas as pd
from pathlib import Path

# ---------- CONFIG ----------
xlsx_path = Path("/Users/koniks/Desktop/Github Folder/public-program-evaluation/data/processed/Delhi_Bus_GTFS.xlsx")
out_dir = xlsx_path.parent
out_xlsx = out_dir / "gtfs_combined.xlsx"
out_csv  = out_dir / "gtfs_combined.csv"

# ---------- LOAD ----------
sheets = pd.read_excel(xlsx_path, sheet_name=None, dtype=str)
sheets = {k.strip().lower(): v for k, v in sheets.items()}

stops = sheets["stops"]
routes = sheets["routes"]
trips = sheets["trips"]
stop_times = sheets["stop_times"]

# ---------- COMBINE ----------
combined = (
    stop_times
    .merge(trips, on="trip_id", how="left")
    .merge(routes, on="route_id", how="left")
    .merge(stops, on="stop_id", how="left")
)

# ---------- SAVE ----------
if len(combined) <= 1_048_576:
    with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
        combined.to_excel(writer, sheet_name="combined", index=False)
else:
    combined.to_csv(out_csv, index=False)

combined.shape


(1048575, 17)

In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path
from shapely.geometry import Point
import warnings
import requests
warnings.filterwarnings('ignore')

print("="*70)
print("DELHI WARD-LEVEL TRANSIT & ECONOMIC ANALYSIS - FIXED VERSION")
print("="*70)

# ---------- CONFIG ----------
base_dir = Path("/Users/koniks/Desktop/Github Folder/public-program-evaluation/data")
gtfs_combined = base_dir / "processed/gtfs_combined.xlsx"
vehicle_file = base_dir / "raw/2021.csv"
pop_file = base_dir / "raw/Census 2011 Ward-Level Population Data (OpenCity).csv"
output_file = base_dir / "clean/analysis_dataset_ward_level.csv"

# ---------- LOAD GTFS DATA ----------
print("\n" + "="*70)
print("1. LOADING GTFS DATA")
print("="*70)

gtfs = pd.read_excel(gtfs_combined, sheet_name='combined', dtype=str)
print(f"✓ Loaded {len(gtfs):,} rows from combined GTFS")

stops = gtfs[['stop_id', 'stop_lat', 'stop_lon']].drop_duplicates()
stops = stops.dropna(subset=['stop_lat', 'stop_lon'])
stops['stop_lat'] = pd.to_numeric(stops['stop_lat'], errors='coerce')
stops['stop_lon'] = pd.to_numeric(stops['stop_lon'], errors='coerce')
stops = stops.dropna()
print(f"✓ Unique stops with coordinates: {len(stops):,}")

# ---------- LOAD SPATIAL BOUNDARIES ----------
print("\n" + "="*70)
print("2. LOADING SPATIAL BOUNDARIES")
print("="*70)

# Load wards
ward_urls = [
    "https://raw.githubusercontent.com/datameet/Municipal_Spatial_Data/master/Delhi/Delhi_Wards.geojson",
    "https://github.com/datameet/Municipal_Spatial_Data/raw/master/Delhi/Delhi_Wards.geojson"
]

print("Downloading Delhi ward boundaries...")
wards = None
for url in ward_urls:
    try:
        wards = gpd.read_file(url)
        print(f"✓ Successfully loaded from: {url}")
        break
    except Exception as e:
        continue

if wards is None:
    raise Exception("Could not load ward boundaries")

print(f"✓ Loaded {len(wards)} municipal wards")

# Create consistent ward keys
wards['ward_id'] = wards['Ward_No'].astype(str).str.strip()
wards['ward_name'] = wards['Ward_Name'].astype(str).str.strip()
wards['ward_no_numeric'] = pd.to_numeric(wards['ward_id'], errors='coerce')

# Create unique ward_key
ward_counts = wards['ward_no_numeric'].value_counts()
non_unique = ward_counts[ward_counts > 1].index
wards['ward_key'] = wards.apply(
    lambda x: x['ward_id'] if pd.isna(x['ward_no_numeric']) or x['ward_no_numeric'] in non_unique.values
    else str(int(x['ward_no_numeric'])),
    axis=1
)
print(f"✓ Created unique ward_key for {len(wards)} wards")

# Load Delhi districts from GADM or datta07 repository
print("Downloading Delhi district boundaries...")
district_urls = [
    "https://raw.githubusercontent.com/datta07/INDIAN-SHAPEFILES/master/STATES/DELHI/DELHI_DISTRICTS.geojson",
    "https://github.com/datta07/INDIAN-SHAPEFILES/raw/master/STATES/DELHI/DELHI_DISTRICTS.geojson"
]

districts = None
for url in district_urls:
    try:
        districts = gpd.read_file(url)
        print(f"✓ Loaded from: {url}")
        break
    except Exception as e:
        continue

if districts is None:
    print("⚠️  Could not load districts, creating from ward spatial join...")
    # Fallback: create districts by aggregating nearby wards
    from sklearn.cluster import KMeans

    ward_centroids = wards.geometry.centroid
    coords = np.array([[p.x, p.y] for p in ward_centroids])

    kmeans = KMeans(n_clusters=11, random_state=42)
    wards['district'] = kmeans.fit_predict(coords)
    wards['district'] = 'District_' + wards['district'].astype(str)
else:
    # Identify district name column
    dist_name_col = None
    for col in ['DISTRICT', 'District', 'district', 'NAME', 'Name', 'name', 'DIST_NAME']:
        if col in districts.columns:
            dist_name_col = col
            break

    if dist_name_col is None:
        dist_name_col = [c for c in districts.columns if c != 'geometry'][0]

    districts['DISTRICT'] = districts[dist_name_col].astype(str).str.strip()
    print(f"✓ Loaded {len(districts)} districts")

    # Spatial join
    wards_with_district = gpd.sjoin(wards, districts[['DISTRICT', 'geometry']],
                                     how='left', predicate='intersects')
    wards_with_district = wards_with_district.drop_duplicates(subset='ward_key')
    wards_with_district['district'] = wards_with_district['DISTRICT'].fillna('Unknown')

    # ✅ ONLY CHANGE: keep ward_no_numeric so later code (NTL seed) doesn't break
    wards = wards_with_district[['ward_key', 'ward_id', 'ward_name', 'ward_no_numeric', 'district', 'geometry']]

print(f"✓ Ward-district mapping complete: {wards['district'].nunique()} unique districts")

# Calculate ward areas
wards['ward_area_km2'] = wards.to_crs('EPSG:32643').geometry.area / 1e6

# ---------- BUS SERVICE METRICS ----------
print("\n" + "="*70)
print("3. CALCULATING BUS SERVICE METRICS")
print("="*70)

stops_gdf = gpd.GeoDataFrame(
    stops,
    geometry=[Point(xy) for xy in zip(stops['stop_lon'], stops['stop_lat'])],
    crs='EPSG:4326'
)

stops_with_ward = gpd.sjoin(stops_gdf, wards, how='left', predicate='within')
matched = stops_with_ward['ward_key'].notna().sum()
print(f"✓ Stops matched to wards: {matched} / {len(stops)}")

service = gtfs.groupby('stop_id').agg({
    'route_id': 'nunique',
    'trip_id': 'count'
}).reset_index()
service.columns = ['stop_id', 'routes_per_stop', 'trips_per_stop']

stops_with_service = stops_with_ward.merge(service, on='stop_id', how='left')
stops_with_service['routes_per_stop'] = stops_with_service['routes_per_stop'].fillna(0)
stops_with_service['trips_per_stop'] = stops_with_service['trips_per_stop'].fillna(0)

ward_metrics = stops_with_service.groupby('ward_key').agg({
    'stop_id': 'count',
    'routes_per_stop': 'sum',
    'trips_per_stop': 'sum'
}).reset_index()
ward_metrics.columns = ['ward_key', 'Stops_i', 'Routes_i', 'TripsPerDay_i']

# Standardize
from scipy.stats import zscore
ward_metrics['zStops'] = zscore(ward_metrics['Stops_i'])
ward_metrics['zRoutes'] = zscore(ward_metrics['Routes_i'])
ward_metrics['zTripsPerDay'] = zscore(ward_metrics['TripsPerDay_i'])
ward_metrics['BusServiceIntensity'] = (
    ward_metrics['zStops'] + ward_metrics['zRoutes'] + ward_metrics['zTripsPerDay']
) / 3

print(f"✓ Bus service metrics calculated for {len(ward_metrics)} wards")

# ---------- POPULATION DATA ----------
print("\n" + "="*70)
print("4. LOADING POPULATION DATA")
print("="*70)

try:
    population = pd.read_csv(pop_file)
    print(f"✓ Loaded population data from local file: {len(population)} rows")
    print(f"  Columns: {population.columns.tolist()}")

    # robust column detection
    cols_l = {c.lower().strip(): c for c in population.columns}
    ward_col = cols_l.get("ward") or cols_l.get("ward_no") or cols_l.get("ward number") or list(population.columns)[0]
    pop_col = cols_l.get("population") or cols_l.get("total population") or cols_l.get("total") or list(population.columns)[1]

    # ward_key from ward label like "DMC Ward No - 192" OR plain numbers
    population["ward_key"] = (
        population[ward_col]
        .astype(str)
        .str.extract(r"(\d+)", expand=False)
        .astype("Int64")
        .astype(str)
    )

    population["Population"] = pd.to_numeric(population[pop_col], errors="coerce")

    # Aggregate duplicates to prevent row inflation during merges
    population = (
        population.dropna(subset=["ward_key"])
        .groupby("ward_key", as_index=False)["Population"].sum()
    )

    print(f"✓ Prepared population for {len(population)} unique ward_key values")

except Exception as e:
    print(f"⚠️  Could not load local population data: {e}")
    population = pd.DataFrame({"ward_key": wards["ward_key"], "Population": np.nan})

# Calculate density
ward_pop = wards[["ward_key", "ward_area_km2"]].merge(population, on="ward_key", how="left")
ward_pop["population_density"] = ward_pop["Population"] / ward_pop["ward_area_km2"]
print("✓ Population density calculated")

# ======================================================================
# ---------- VEHICLE OWNERSHIP FROM RTO DATA (REWRITTEN: NEAREST-RTO) ----
# ======================================================================
print("\n" + "="*70)
print("5. PROCESSING VEHICLE OWNERSHIP (RTO DATA) — NEAREST RTO ASSIGNMENT")
print("="*70)

# ---- Load vehicle data ----
vehicle_raw = pd.read_csv(vehicle_file)
print(f"✓ Loaded vehicle data: {len(vehicle_raw)} rows")
print(f"  Columns: {vehicle_raw.columns.tolist()}")

# ---- Robust column detection ----
def find_col(df, candidates):
    cols_l = {c.lower().strip(): c for c in df.columns}
    for cand in candidates:
        if cand.lower().strip() in cols_l:
            return cols_l[cand.lower().strip()]
    # contains fallback
    for cand in candidates:
        for c in df.columns:
            if cand.lower() in c.lower():
                return c
    return None

rto_id_col   = find_col(vehicle_raw, ["rto"])
rto_name_col = find_col(vehicle_raw, ["rto name"])
count_col    = find_col(vehicle_raw, ["count"])

veh_type_col = find_col(vehicle_raw, ["name"])

# Require minimal inputs
if rto_id_col is None and rto_name_col is None:
    raise ValueError("Could not find RTO identifier column in 2021.csv (expected something like 'RTO' or 'RTO Name').")
if count_col is None:
    raise ValueError("Could not find the vehicle count column in 2021.csv (expected something like 'Count').")
if veh_type_col is None:
    raise ValueError(
        "Could not find a vehicle-type column in 2021.csv.\n"
        "To filter out tractors/bicycles and keep 2W/4W, the file must have a type/class/category column."
    )

print("Detected columns:")
print("  rto_id_col  :", rto_id_col)
print("  rto_name_col:", rto_name_col)
print("  veh_type_col:", veh_type_col)
print("  count_col   :", count_col)

# Create unified RTO label
vehicle_raw["RTO_LABEL"] = None
if rto_name_col is not None:
    vehicle_raw["RTO_LABEL"] = vehicle_raw[rto_name_col].astype(str).str.strip()
elif rto_id_col is not None:
    vehicle_raw["RTO_LABEL"] = vehicle_raw[rto_id_col].astype(str).str.strip()
else:
    # both present? prefer name, but keep id as fallback
    vehicle_raw["RTO_LABEL"] = (vehicle_raw[rto_id_col].astype(str).str.strip() + " - " +
                                vehicle_raw[rto_name_col].astype(str).str.strip())

# ---- Filter vehicle types: KEEP 2W + 4W, EXCLUDE tractors/bicycles/etc. ----
vehicle_raw["veh_type_norm"] = vehicle_raw[veh_type_col].astype(str).str.upper().str.strip()

# Keep keywords (tune if your file uses different naming)
KEEP_PATTERNS = [
    "TWO WHEELER", "2W", "MOTORCYCLE", "SCOOTER", "MOPED",
    "MOTOR CAR", "CAR", "JEEP", "VAN", "FOUR WHEELER", "4W",
    "LMV"  # sometimes used for light motor vehicle
]
DROP_PATTERNS = [
    "TRACTOR", "BICYCLE", "CYCLE", "TRAILER", "HANDCART", "BULLOCK",
    "CART", "ANIMAL", "HARVEST", "THRESH", "FORKLIFT",
    "CONSTRUCTION", "EXCAVATOR", "CRANE"
]

keep_mask = vehicle_raw["veh_type_norm"].apply(lambda s: any(k in s for k in KEEP_PATTERNS))
drop_mask = vehicle_raw["veh_type_norm"].apply(lambda s: any(k in s for k in DROP_PATTERNS))

vehicle_filt = vehicle_raw[keep_mask & ~drop_mask].copy()
vehicle_filt[count_col] = pd.to_numeric(vehicle_filt[count_col], errors="coerce")
vehicle_filt = vehicle_filt.dropna(subset=[count_col])

print(f"✓ Retained {len(vehicle_filt):,} rows after vehicle-type filtering")
print("  Sample kept vehicle types:", vehicle_filt["veh_type_norm"].value_counts().head(8).index.tolist())

# ---- Aggregate to RTO-level ownership score ----
# Here "ownership_score" is total (2W+4W) registrations per RTO (sum across retained types).
rto_ownership = (
    vehicle_filt.groupby("RTO_LABEL", as_index=False)[count_col]
    .sum()
    .rename(columns={count_col: "vehicle_ownership_rto"})
)

print(f"✓ Computed ownership score for {len(rto_ownership)} unique RTO offices")
display(rto_ownership.sort_values("vehicle_ownership_rto", ascending=False).head(10))

# ---- RTO office locations ----
# IMPORTANT: We will assign EACH WARD to the NEAREST RTO office by distance (metric CRS),
# and then use that RTO's ownership score for the ward.
rto_locations = pd.DataFrame({
    'RTO': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'RTO_LABEL': ['MALL ROAD', 'IP DEPOT', 'SARAI KALE KHAN', 'JANAKPURI',
                  'LONI ROAD', 'MAYUR VIHAR', 'WAZIRPUR', 'DWARKA',
                  'RAJA GARDEN', 'ROHINI', 'VASANT VIHAR', 'SURAJMAL VIHAR',
                  'HARI NAGAR', 'EAST', 'SOUTH', 'NORTH', 'WEST',
                  'CENTRAL', 'NEW DELHI', 'SHAHDARA'],
    'lat': [28.7041, 28.6510, 28.5535, 28.6219, 28.7383, 28.6084, 28.7199,
            28.5921, 28.6503, 28.7199, 28.5494, 28.6509, 28.6215, 28.6510,
            28.5535, 28.7041, 28.6503, 28.6431, 28.6139, 28.6843],
    'lon': [77.1025, 77.2526, 77.2577, 77.0807, 77.3184, 77.3051, 77.1674,
            77.0460, 77.1272, 77.0674, 77.2001, 77.3389, 77.0524, 77.2526,
            77.2577, 77.1025, 77.1272, 77.2197, 77.2090, 77.2749]
})
print(f"✓ Using {len(rto_locations)} RTO office locations (fixed list)")

# ---- Match RTO ownership table to location table ----
# Vehicle file may use slightly different RTO names than the fixed list.
# We'll fuzzy match.
from difflib import get_close_matches

def fuzzy_match_name(name, choices, cutoff=0.6):
    matches = get_close_matches(str(name).upper(), [c.upper() for c in choices], n=1, cutoff=cutoff)
    if matches:
        # return original choice (preserve exact capitalization from choices)
        m_upper = matches[0]
        for c in choices:
            if c.upper() == m_upper:
                return c
    return None

choices = rto_locations["RTO_LABEL"].tolist()
rto_ownership["RTO_LABEL_MATCHED"] = rto_ownership["RTO_LABEL"].apply(lambda x: fuzzy_match_name(x, choices, cutoff=0.6))
matched_share = rto_ownership["RTO_LABEL_MATCHED"].notna().mean()
print(f"✓ Fuzzy-matched {matched_share*100:.1f}% of RTO labels to the location list")

rto_ref = (
    rto_ownership.dropna(subset=["RTO_LABEL_MATCHED"])
    .merge(rto_locations, left_on="RTO_LABEL_MATCHED", right_on="RTO_LABEL", how="inner")
)

if len(rto_ref) == 0:
    raise ValueError(
        "No RTO offices matched between vehicle data and rto_locations list.\n"
        "You likely need to update rto_locations['RTO_LABEL'] to match your 2021.csv naming."
    )

# ---- Create RTO GeoDataFrame ----
rto_gdf = gpd.GeoDataFrame(
    rto_ref,
    geometry=[Point(xy) for xy in zip(rto_ref["lon"], rto_ref["lat"])],
    crs="EPSG:4326"
)

# ---- Compute ward centroids and assign nearest RTO (metric CRS) ----
# Use a projected CRS for accurate distance (meters): EPSG:32643 works for Delhi region.
wards_utm = wards.to_crs("EPSG:32643").copy()
wards_utm["centroid"] = wards_utm.geometry.centroid
wards_cent = wards_utm.set_geometry("centroid")

rto_utm = rto_gdf.to_crs("EPSG:32643").copy()

# Efficient nearest-neighbor join (GeoPandas >= 0.10 supports sjoin_nearest)
try:
    ward_to_rto = gpd.sjoin_nearest(
        wards_cent[["ward_key", "centroid"]],
        rto_utm[["RTO_LABEL_MATCHED", "vehicle_ownership_rto", "geometry"]],
        how="left",
        distance_col="dist_to_rto_m"
    )
except Exception as e:
    # Fallback if sjoin_nearest isn't available
    print("⚠️  sjoin_nearest not available, using manual nearest computation. Reason:", e)
    ward_to_rto = wards_cent[["ward_key", "centroid"]].copy()
    ward_to_rto["RTO_LABEL_MATCHED"] = None
    ward_to_rto["vehicle_ownership_rto"] = np.nan
    ward_to_rto["dist_to_rto_m"] = np.nan

    # brute force (OK for ~290 wards and ~20 RTOs)
    for i, w in ward_to_rto.iterrows():
        dists = rto_utm.geometry.distance(w["centroid"])
        j = int(dists.argmin())
        ward_to_rto.at[i, "RTO_LABEL_MATCHED"] = rto_utm.iloc[j]["RTO_LABEL_MATCHED"]
        ward_to_rto.at[i, "vehicle_ownership_rto"] = rto_utm.iloc[j]["vehicle_ownership_rto"]
        ward_to_rto.at[i, "dist_to_rto_m"] = float(dists.iloc[j])

# Keep only is needed
vehicle_per_ward = ward_to_rto[["ward_key", "vehicle_ownership_rto", "dist_to_rto_m", "RTO_LABEL_MATCHED"]].copy()
vehicle_per_ward = vehicle_per_ward.rename(columns={
    "vehicle_ownership_rto": "vehicle_ownership",
    "RTO_LABEL_MATCHED": "nearest_rto"
})

print(f"✓ Assigned nearest RTO ownership to {vehicle_per_ward['vehicle_ownership'].notna().sum()} wards (should be all wards)")
display(vehicle_per_ward.head())

# ---------- NIGHTTIME LIGHTS DATA ----------
print("\n" + "="*70)
print("6. PROCESSING NIGHTTIME LIGHTS (VIIRS 2012)")
print("="*70)

try:
    import rasterio
    from rasterio.mask import mask
    import warnings
    warnings.filterwarnings('ignore', category=rasterio.errors.NotGeoreferencedWarning)
    
    ntl_file = base_dir / "raw/VNL_v21_npp_201204-201212_global_vcmcfg_c202205302300.average.dat.tif" 
    
    # Load the raster data
    with rasterio.open(ntl_file) as src:
        ntl_crs = src.crs
        print(f"  Raster CRS: {ntl_crs}")
        print(f"  Raster shape: {src.shape}")
        print(f"  Raster bounds: {src.bounds}")
        
        # Reproject wards to match raster CRS if needed
        wards_ntl = wards.to_crs(ntl_crs) if wards.crs != ntl_crs else wards.copy()
        
        # Extract nighttime lights statistics for each ward
        ntl_stats = []
        
        print(f"Extracting nighttime lights for {len(wards_ntl)} wards...")
        for idx, ward in wards_ntl.iterrows():
            try:
                # Mask raster to ward boundary
                ward_geom = [ward.geometry.__geo_interface__]
                out_image, out_transform = mask(src, ward_geom, crop=True, nodata=0)
                
                # Extract radiance values (note: values are already multiplied by 1E9)
                # Units: nanoWatts/cm²/sr
                radiance_values = out_image[0]
                
                # Filter out zero/nodata values
                valid_values = radiance_values[radiance_values > 0]
                
                if len(valid_values) > 0:
                    # Calculate statistics
                    mean_radiance = float(valid_values.mean())
                    median_radiance = float(np.median(valid_values))
                    total_radiance = float(valid_values.sum())
                    max_radiance = float(valid_values.max())
                    lit_pixels = int(len(valid_values))
                else:
                    mean_radiance = 0.0
                    median_radiance = 0.0
                    total_radiance = 0.0
                    max_radiance = 0.0
                    lit_pixels = 0
                
                ntl_stats.append({
                    'ward_key': ward['ward_key'],
                    'nighttime_lights': mean_radiance,  # Primary metric
                    'ntl_median': median_radiance,
                    'ntl_total': total_radiance,
                    'ntl_max': max_radiance,
                    'ntl_lit_pixels': lit_pixels
                })
                
            except Exception as e:
                print(f"  ⚠️  Error processing ward {ward['ward_key']}: {e}")
                ntl_stats.append({
                    'ward_key': ward['ward_key'],
                    'nighttime_lights': np.nan,
                    'ntl_median': np.nan,
                    'ntl_total': np.nan,
                    'ntl_max': np.nan,
                    'ntl_lit_pixels': 0
                })
        
        ntl_df = pd.DataFrame(ntl_stats)
        
        print(f"✓ Nighttime lights extracted for {len(ntl_df)} wards")
        print(f"  Coverage: {(ntl_df['nighttime_lights'] > 0).sum()} wards with detected lights")
        print(f"  Mean radiance: {ntl_df['nighttime_lights'].mean():.2f} nW/cm²/sr")
        print(f"  Median radiance: {ntl_df['nighttime_lights'].median():.2f} nW/cm²/sr")
        print(f"  Max radiance: {ntl_df['nighttime_lights'].max():.2f} nW/cm²/sr")

except ImportError:
    print("⚠️  rasterio not installed. Installing required package...")
    print("Run: pip install rasterio")
    # Create empty dataframe with NaN values
    ntl_df = pd.DataFrame({
        'ward_key': wards['ward_key'],
        'nighttime_lights': np.nan,
        'ntl_median': np.nan,
        'ntl_total': np.nan,
        'ntl_max': np.nan,
        'ntl_lit_pixels': 0
    })

except FileNotFoundError as e:
    print(f"⚠️  {e}")
    # Create empty dataframe with NaN values
    ntl_df = pd.DataFrame({
        'ward_key': wards['ward_key'],
        'nighttime_lights': np.nan,
        'ntl_median': np.nan,
        'ntl_total': np.nan,
        'ntl_max': np.nan,
        'ntl_lit_pixels': 0
    })

except Exception as e:
    print(f"⚠️  Error processing nighttime lights: {e}")
    import traceback
    traceback.print_exc()
    # Create empty dataframe with NaN values
    ntl_df = pd.DataFrame({
        'ward_key': wards['ward_key'],
        'nighttime_lights': np.nan,
        'ntl_median': np.nan,
        'ntl_total': np.nan,
        'ntl_max': np.nan,
        'ntl_lit_pixels': 0
    })

# ---------- INFRASTRUCTURE METRICS ----------
print("\n" + "="*70)
print("7. CALCULATING INFRASTRUCTURE METRICS")
print("="*70)

# Metro stations (expanded list)
metro_stations = pd.DataFrame({
    'name': ['AIIMS', 'Adarsh Nagar', 'Akshardham', 'Anand Vihar ISBT', 'Arjan Garh', 'Arthala', 'Ashok Park Main', 'Ashram', 'Azadpur', 'Badarpur Border', 'Bahadurgarh City', 'Barakhambha Road', 'Bata Chowk', 'Bhikaji Cama Place', 'Botanical Garden', 'Brigadier Hoshiyar Singh', 'Central Secretariat', 'Chandni Chowk', 'Chawri Bazar', 'Chhatarpur', 'Chirag Delhi', 'Civil Lines', 'Dabri Mor-Janakpuri South', 'Dashrath Puri', 'Delhi Aerocity', 'Delhi Cantonment', 'Delhi Gate', 'Dhaula Kuan', 'Dilshad Garden', 'Durgabai Deshmukh South Campus', 'Dwarka', 'Dwarka Mor', 'Dwarka Sector 10', 'Dwarka Sector 11', 'Dwarka Sector 12', 'Dwarka Sector 13', 'Dwarka Sector 14', 'Dwarka Sector 21', 'Dwarka Sector 8', 'Dwarka Sector 9', 'ESI Hospital', 'East Azad Nagar', 'East Vinod Nagar – Mayur Vihar-II', 'Escorts Mujesar', 'GTB Nagar', 'Ghevra', 'Ghitorni', 'Gokulpuri', 'Golf Course', 'Govind Puri', 'Greater Kailash', 'Green Park', 'Guru Dronacharya', 'HUDA City Centre', 'Haiderpur', 'Harkesh Nagar Okhla', 'Hauz Khas', 'Hazrat Nizamuddin', 'Hindon', 'IFFCO Chowk', 'INA', 'IP Extension', 'ITO', 'Inderlok', 'Indira Gandhi International Airport', 'Indraprastha', 'Jaffrabad', 'Jahangirpuri', 'Jama Masjid', 'Jamia Millia Islamia', 'Janakpuri East', 'Janakpuri West', 'Jangpura', 'Janpath', 'Jasola Apollo', 'Jasola Vihar Shaheen Bagh', 'Jawaharlal Nehru Stadium', 'Jhandewalan', 'Jhilmil', 'Jor Bagh', 'Kailash Colony', 'Kalindi Kunj', 'Kalkaji Mandir', 'Kanhaiya Nagar', 'Karkarduma', 'Karkarduma Court', 'Karol Bagh', 'Kashmere Gate', 'Kaushambi', 'Keshav Puram', 'Khan Market', 'Kirti Nagar†', 'Kohat Enclave', 'Krishna Nagar', 'Lajpat Nagar', 'Lal Qila', 'Laxmi Nagar', 'Lok Kalyan Marg', 'MG Road', 'Madipur', 'Majlis Park', 'Major Mohit Sharma', 'Malviya Nagar', 'Mandawali – West Vinod Nagar', 'Mandi House', 'Mansarovar Park', 'Maujpur-Babarpur', 'Mayapuri', 'Mayur Vihar -I', 'Mayur Vihar Extension', 'Mayur Vihar Pocket I', 'Mewala Maharajpur', 'Model Town', 'Mohan Estate', 'Mohan Nagar', 'Moolchand', 'Moti Nagar', 'Mundka', 'Mundka Industrial Area', 'Munirka', 'Najafgarh', 'Nangli', 'Nangloi', 'Nangloi Railway station', 'Naraina Vihar', 'Nawada', 'Neelam Chowk Ajronda', 'Nehru Enclave', 'Nehru Place', 'Netaji Subhash Place', 'New Ashok Nagar', 'New Delhi', 'Nirman Vihar', 'Noida Electronic City', 'Noida Sector 15', 'Noida Sector 16', 'Noida Sector 18', 'Noida Sector 34', 'Noida Sector 52', 'Noida Sector 59', 'Noida Sector 61', 'Noida Sector 62', 'Okhla Bird Sanctuary', 'Okhla NSIC', 'Okhla Vihar', 'Old Faridabad', 'Palam', 'Panchsheel Park', 'Pandit Shree Ram Sharma', 'Paschim Vihar East', 'Paschim Vihar West', 'Patel Chowk', 'Patel Nagar', 'Peera Garhi', 'Pitam Pura', 'Pratap Nagar', 'Preet Vihar', 'Pul Bangash', 'Punjabi Bagh', 'Punjabi Bagh West', 'R.K.Puram', 'Raj Bagh', 'Raja Nahar Singh', 'Rajdhani Park', 'Rajendra Place', 'Rajiv Chowk', 'Rajouri Garden', 'Ramakrishna Ashram Marg', 'Ramesh Nagar', 'Rithala', 'Rohini East', 'Rohini Sector 18', 'Rohini West', 'Sadar Bazaar Cantonment', 'Saket', 'Samaypur Badli', 'Sant Surdas (Sihi)', 'Sarai', 'Sarita Vihar', 'Sarojini Nagar', 'Satguru Ramsingh Marg', 'Sector 28', 'Seelampur', 'Shadipur', 'Shahdara', 'Shaheed Nagar', 'Shaheed Sthal', 'Shakurpur', 'Shalimar Bagh', 'Shankar Vihar', 'Shastri Nagar', 'Shastri Park', 'Shiv Vihar', 'Shivaji Park', 'Shivaji Stadium', 'Shyam Park', 'Sikandarpur', 'Sir Vishweshwaraiah Moti Bagh', 'South', 'Subhash Nagar', 'Sukhdev Vihar', 'Sultanpur', 'Supreme Court', 'Tagore Garden', 'Tikri Border', 'Tikri Kalan', 'Tilak Nagar', 'Tis Hazari', 'Trilokpuri Sanjay Lake', 'Tughlakabad Station', 'Udyog Bhawan', 'Udyog Nagar', 'Uttam Nagar East', 'Uttam Nagar West', 'Vaishali', 'Vasant Vihar', 'Vidhan Sabha', 'Vinobapuri', 'Vishwa Vidyalaya', 'Welcome', 'Yamuna Bank'],
    'lat': [28.56686, 28.714401, 28.617842, 28.646753, 28.480735, 28.676999, 28.671604, 28.572423, 28.707657, 28.4905, 28.690785, 28.629768, 28.385836, 28.5679, 28.563896, 28.69746, 28.615879, 28.660504, 28.65016, 28.506724, 28.538141, 28.676851, 28.615756, 28.601875, 28.548798, 28.593833, 28.639204, 28.59189, 28.675886, 28.589438, 28.577192, 28.61931, 28.581058, 28.586447, 28.592236, 28.597009, 28.602253, 28.551838, 28.565611, 28.574272, 28.658074, 28.664696, 28.620044, 28.370234, 28.698132, 28.685238, 28.493751, 28.702475, 28.597781, 28.544377, 28.541878, 28.558582, 28.482021, 28.459343, 28.730121, 28.543125, 28.544256, 28.588749, 28.878965, 28.472328, 28.574408, 28.628899, 28.630509, 28.651718, 28.554897, 28.620602, 28.682682, 28.725972, 28.65001, 28.55849, 28.633021, 28.62895, 28.582457, 28.60886, 28.538425, 28.545828, 28.583377, 28.644319, 28.675703, 28.588239, 28.553052, 28.545219, 28.549775, 28.680064, 28.649162, 28.6536, 28.644187, 28.651718, 28.645329, 28.688926, 28.600135, 28.653281, 28.698042, 28.657846, 28.570602, 17.920862, 28.630555, 28.598385, 28.479458, 28.676475, 28.724431, 28.677611, 28.53392, 28.624971, 28.625556, 28.675739, 28.691978, 28.63718, 28.603133, 28.594158, 28.605862, 28.441875, 28.702714, 28.51959, 28.606319, 28.564404, 28.657859, 28.682434, 28.683449, 28.554886, 28.612304, 28.6173, 28.682401, 28.682201, 28.627338, 28.620276, 28.397482, 28.546058, 28.549257, 28.696052, 28.589088, 28.643642, 28.636506, 28.579354, 28.480863, 28.480863, 28.480863, 28.480863, 28.480863, 28.480863, 28.480863, 28.480863, 28.552942, 28.554483, 28.5613, 28.480863, 28.591893, 28.543352, 28.689281, 28.677398, 28.678651, 28.622966, 28.645586, 28.679807, 28.703268, 28.66649, 28.641495, 28.666375, 28.668945, 28.67032, 28.551426, 28.64086, 28.340019, 28.682308, 28.642496, 28.63278, 28.642152, 28.639244, 28.652744, 28.720806, 28.70759, 28.738348, 28.714957, 28.577151, 28.524411, 28.744616, 28.354651, 28.651718, 28.52866, 28.574158, 28.661933, 28.545257, 28.66983, 28.651636, 28.673404, 28.53078, 28.670611, 28.685767, 28.717453, 28.557439, 28.669975, 28.668278, 28.617538, 28.675032, 28.631509, 28.698807, 28.48127, 28.578533, 28.651718, 28.641807, 28.559748, 28.461658, 28.622288, 28.643764, 28.688025, 28.686866, 28.636548, 28.667132, 28.613453, 28.50236, 28.61053, 28.681103, 28.624846, 28.621757, 28.644091, 28.560691, 28.686322, 28.566976, 28.695037, 28.651718, 28.623278],
    'lon': [77.207806, 77.167288, 77.279488, 77.318004, 77.125762, 77.391892, 77.155291, 77.258598, 77.175547, 77.304038, 76.935485, 77.225005, 77.313462, 77.187016, 77.334332, 76.919203, 77.212282, 77.22978, 77.229501, 77.175002, 77.228069, 77.22503, 77.085178, 77.082356, 77.120809, 77.134979, 77.240782, 77.161703, 77.32144, 77.169082, 77.044294, 77.033279, 77.0575, 77.049438, 77.040732, 77.033541, 77.025957, 77.058649, 77.067037, 77.065332, 77.127268, 77.284881, 77.305407, 77.31492, 77.206411, 76.996159, 77.149187, 77.286125, 77.159817, 77.264266, 77.238455, 77.206718, 77.102268, 77.072657, 77.149403, 77.275562, 77.206707, 77.257249, 77.415483, 77.072422, 77.210241, 77.310198, 77.241436, 77.221939, 77.084675, 77.249551, 77.274806, 77.162658, 77.237676, 77.281165, 77.086757, 77.077924, 77.2415, 77.218165, 77.283038, 77.296658, 77.23354, 77.199916, 77.312505, 77.216528, 77.242969, 77.305989, 77.260667, 77.164844, 77.306208, 77.295788, 77.189132, 77.221939, 77.324348, 77.161683, 77.226491, 77.141773, 77.140539, 77.290185, 77.23653, 77.528502, 77.277633, 77.202996, 77.080465, 77.11936, 77.181964, 77.358143, 77.212447, 77.304491, 77.234195, 77.300102, 77.279624, 77.129733, 77.292589, 77.294589, 77.298702, 77.3023, 77.193991, 77.294585, 77.106082, 77.234278, 77.142429, 77.030574, 77.017133, 77.171084, 76.982391, 77.010438, 77.064732, 77.056246, 77.140318, 77.045079, 77.31236, 77.251506, 77.252952, 77.15264, 77.301508, 77.221737, 77.286835, 77.293303, 77.084888, 77.084888, 77.084888, 77.084888, 77.084888, 77.084888, 77.084888, 77.084888, 77.321595, 77.264849, 77.29193, 77.084888, 77.082824, 77.214076, 76.951199, 77.112295, 77.102226, 77.214031, 77.168698, 77.092679, 77.13225, 77.199336, 77.295374, 77.207529, 77.132461, 77.142088, 77.184701, 77.2095, 77.316428, 77.043944, 77.178276, 77.2197, 77.11606, 77.208579, 77.13165, 77.107181, 77.126039, 77.139832, 77.115385, 77.111153, 77.213725, 77.138265, 77.316226, 77.221939, 77.288397, 77.19537, 77.157477, 77.032576, 77.266754, 77.158295, 77.28974, 77.212057, 77.415582, 77.149609, 77.150867, 77.139665, 77.182, 77.250399, 77.03543, 77.130472, 77.216059, 77.268464, 77.092999, 77.175741, 77.221939, 77.108761, 77.2749, 76.870085, 77.239434, 77.112845, 76.964083, 76.977207, 77.096496, 77.216628, 77.308855, 77.29924, 77.213108, 77.080849, 77.065286, 77.055712, 77.068386, 77.160791, 77.221727, 77.249191, 77.214719, 77.221939, 77.267924],
})

metro_gdf = gpd.GeoDataFrame(
    metro_stations,
    geometry=[Point(xy) for xy in zip(metro_stations['lon'], metro_stations['lat'])],
    crs='EPSG:4326'
)

# Calculate minimum distance to metro (in km)
ward_centroids = wards.to_crs('EPSG:32643').copy()
ward_centroids['centroid'] = ward_centroids.geometry.centroid
ward_centroids = ward_centroids.set_geometry('centroid')

metro_utm = metro_gdf.to_crs('EPSG:32643')

metro_access = []
for idx, ward in ward_centroids.iterrows():
    distances = metro_utm.geometry.distance(ward.geometry) / 1000  # Convert to km
    min_distance_km = distances.min()
    metro_access.append({
        'ward_key': ward['ward_key'],
        'metro_access_km': min_distance_km
    })

metro_access_df = pd.DataFrame(metro_access)
print(f"✓ Metro access calculated for {len(metro_access_df)} wards")
print(f"  Mean distance: {metro_access_df['metro_access_km'].mean():.2f} km")

# Road density
try:
    import osmnx as ox
    print("Downloading road network from OpenStreetMap...")

    roads = ox.graph_from_place('Delhi, India', network_type='drive')
    roads_gdf = ox.graph_to_gdfs(roads, nodes=False)
    roads_gdf = roads_gdf.to_crs('EPSG:32643')

    wards_utm = wards.to_crs('EPSG:32643')
    road_density = []

    for idx, ward in wards_utm.iterrows():
        ward_roads = roads_gdf[roads_gdf.intersects(ward.geometry)]
        total_length = ward_roads.geometry.length.sum() / 1000
        density = total_length / ward['ward_area_km2'] if ward['ward_area_km2'] > 0 else 0
        road_density.append({
            'ward_key': ward['ward_key'],
            'road_density_km_per_km2': density
        })

    road_density_df = pd.DataFrame(road_density)
    print(f"✓ Road density calculated for {len(road_density_df)} wards")

except Exception as e:
    print(f"⚠️  Could not calculate road density: {e}")
    road_density_df = pd.DataFrame({
        'ward_key': wards['ward_key'],
        'road_density_km_per_km2': np.nan
    })

# ---------- DISTRICT FIXED EFFECTS ----------
print("\n" + "="*70)
print("8. CREATING DISTRICT FIXED EFFECTS")
print("="*70)

district_dummies = pd.get_dummies(wards['district'], prefix='district')
district_dummies['ward_key'] = wards['ward_key'].values
print(f"✓ Created {len(district_dummies.columns)-1} district dummy variables")

# ======================================================================
# -------------------------- FINAL MERGE (REWRITTEN) --------------------
# ======================================================================
print("\n" + "="*70)
print("9. MERGING ALL DATA SOURCES — WITH NEAREST-RTO VEHICLE OWNERSHIP")
print("="*70)

final_df = wards[['ward_key', 'ward_id', 'ward_name', 'district', 'ward_area_km2']].copy()

final_df = final_df.merge(ward_metrics, on='ward_key', how='left')
final_df = final_df.merge(ward_pop[['ward_key', 'Population', 'population_density']], on='ward_key', how='left')
final_df = final_df.merge(vehicle_per_ward, on='ward_key', how='left')
final_df = final_df.merge(ntl_df, on='ward_key', how='left')
final_df = final_df.merge(metro_access_df, on='ward_key', how='left')
final_df = final_df.merge(road_density_df, on='ward_key', how='left')
final_df = final_df.merge(district_dummies, on='ward_key', how='left')

print(f" Final dataset: {len(final_df)} rows × {len(final_df.columns)} columns")
print(f"  Vehicle ownership coverage: {(final_df['vehicle_ownership'].notna().mean()*100):.1f}%")
print(f"  (Optional) Nearest RTO label coverage: {(final_df['nearest_rto'].notna().mean()*100):.1f}%")


# ---------- DATA QUALITY CHECKS ----------
print("\n" + "="*70)
print("10. DATA QUALITY CHECKS")
print("="*70)

missing = final_df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if len(missing) > 0:
    print(f"Missing value summary:")
    print(missing.head(10))
else:
    print("✓ No missing values!")

checks = {
    'Ward keys unique': len(final_df['ward_key'].unique()) == len(final_df),
    'Bus service coverage': f"{(final_df['BusServiceIntensity'].notna().sum() / len(final_df) * 100):.1f}%",
    'Population coverage': f"{(final_df['Population'].notna().sum() / len(final_df) * 100):.1f}%",
    'Vehicle ownership coverage': f"{(final_df['vehicle_ownership'].notna().sum() / len(final_df) * 100):.1f}%",
    'Nighttime lights coverage': f"{(final_df['nighttime_lights'].notna().sum() / len(final_df) * 100):.1f}%",
    'Metro access coverage': f"{(final_df['metro_access_km'].notna().sum() / len(final_df) * 100):.1f}%"
}

for check, value in checks.items():
    print(f"  {check}: {value}")

# ---------- SAVE OUTPUT ----------
print("\n" + "="*70)
print("11. SAVING OUTPUT")
print("="*70)

final_df.to_csv(output_file, index=False)
print(f"✓ Dataset saved: {output_file}")
print(f"\nFinal shape: {final_df.shape}")

# Summary statistics
print(f"\n{'='*70}")
print("SUMMARY STATISTICS")
print(f"{'='*70}")

numeric_cols = ['BusServiceIntensity', 'Population', 'vehicle_ownership',
                'nighttime_lights', 'metro_access_km', 'population_density']

for col in numeric_cols:
    if col in final_df.columns and final_df[col].notna().sum() > 0:
        print(f"\n{col}:")
        print(f"  Mean: {final_df[col].mean():.2f}")
        print(f"  Std: {final_df[col].std():.2f}")
        print(f"  Min: {final_df[col].min():.2f}")
        print(f"  Max: {final_df[col].max():.2f}")
        print(f"  Missing: {final_df[col].isna().sum()} ({(final_df[col].isna().sum()/len(final_df)*100):.1f}%)")

print(f"\n{'='*70}")
print("✓ ANALYSIS COMPLETE!")
print(f"{'='*70}")
print("\nFirst 5 rows:")
print(final_df[['ward_key', 'ward_name', 'Population', 'vehicle_ownership',
                'nighttime_lights', 'BusServiceIntensity']].head())


DELHI WARD-LEVEL TRANSIT & ECONOMIC ANALYSIS - FIXED VERSION

1. LOADING GTFS DATA
✓ Loaded 1,048,575 rows from combined GTFS
✓ Unique stops with coordinates: 4,297

2. LOADING SPATIAL BOUNDARIES
✓ Successfully loaded from: https://raw.githubusercontent.com/datameet/Municipal_Spatial_Data/master/Delhi/Delhi_Wards.geojson
✓ Loaded 290 municipal wards
✓ Created unique ward_key for 290 wards
✓ Loaded from: https://raw.githubusercontent.com/datta07/INDIAN-SHAPEFILES/master/STATES/DELHI/DELHI_DISTRICTS.geojson
✓ Loaded 11 districts
✓ Ward-district mapping complete: 11 unique districts

3. CALCULATING BUS SERVICE METRICS
✓ Stops matched to wards: 4289 / 4297
✓ Bus service metrics calculated for 255 wards

4. LOADING POPULATION DATA
✓ Loaded population data from local file: 289 rows
  Columns: ['_id', 'Sl No', 'Ward', 'Population']
✓ Prepared population for 217 unique ward_key values
✓ Population density calculated

5. PROCESSING VEHICLE OWNERSHIP (RTO DATA) — NEAREST RTO ASSIGNMENT
✓ Loaded 

,RTO_LABEL,vehicle_ownership_rto
0,BURARI AUTO UNIT,1734817
6,MALL ROAD,770042
1,BURARI TAXI UNIT,561323
9,RAJPUR ROAD/VIU BURARI,164142
5,LONI ROAD,116398
15,WAZIRPUR,80675
12,SOUTH DELHI,80103
2,DWARKA,53393
11,SARAI KALE KHAN,42362
10,ROHINI,35604


✓ Using 20 RTO office locations (fixed list)
✓ Fuzzy-matched 75.0% of RTO labels to the location list
✓ Assigned nearest RTO ownership to 322 wards (should be all wards)


,ward_key,vehicle_ownership,dist_to_rto_m,nearest_rto
0,CANT_1,29352,3233.226766,RAJA GARDEN
1,CANT_2,29352,4661.855167,RAJA GARDEN
2,CANT_4,33224,5551.821985,VASANT VIHAR
3,CANT_5,31757,7113.443040,JANAKPURI
4,CANT_6,31757,6295.381595,JANAKPURI



6. PROCESSING NIGHTTIME LIGHTS (VIIRS 2012)
  Raster CRS: EPSG:4326
  Raster shape: (33601, 86401)
  Raster bounds: BoundingBox(left=-180.00208333335, bottom=-65.00208445335001, right=180.00208621335, top=75.00208333335)
Extracting nighttime lights for 290 wards...
✓ Nighttime lights extracted for 290 wards
  Coverage: 290 wards with detected lights
  Mean radiance: 50.04 nW/cm²/sr
  Median radiance: 53.30 nW/cm²/sr
  Max radiance: 95.98 nW/cm²/sr

7. CALCULATING INFRASTRUCTURE METRICS
✓ Metro access calculated for 290 wards
  Mean distance: 0.51 km
✓ Road density calculated for 290 wards

8. CREATING DISTRICT FIXED EFFECTS
✓ Created 11 district dummy variables

9. MERGING ALL DATA SOURCES — WITH NEAREST-RTO VEHICLE OWNERSHIP
 Final dataset: 322 rows × 35 columns
  Vehicle ownership coverage: 100.0%
  (Optional) Nearest RTO label coverage: 100.0%

10. DATA QUALITY CHECKS
Missing value summary:
Population             77
population_density     77
Stops_i                41
Routes_i      